In [82]:
import pandas as pd
import numpy as np
import os
import string
from tqdm.notebook import tqdm
import tensorflow as tf

In [78]:
tf.__version__

'2.1.1'

In [7]:
data_dir = 'dataset/'
file_names = [data_dir + i for i in os.listdir(data_dir)]
print(file_names[:5])

['dataset/yob1880.txt', 'dataset/yob1881.txt', 'dataset/yob1882.txt', 'dataset/yob1883.txt', 'dataset/yob1884.txt']


In [17]:
name_counts_dict = {}
total_names = 0

for fname in file_names:
    with open(fname) as f:
        lines = f.readlines()
        for line in lines:
            name, gender, count = line.split(',')
            count = int(count.strip('\n'))
            if gender.upper() == "F":
                if name in name_counts_dict:
                    name_counts_dict[name] += count
                else:
                    name_counts_dict[name] = count
                total_names += count

            

In [23]:
df = pd.DataFrame(name_counts_dict.items(), columns=['Name', 'Count'])

In [31]:
df = df.sort_values(by="Count", ascending=False)
df['Normalized_Count'] = df.Count / total_names
df.head()

,Name,Count,Normalized_Count
0,Mary,4128052,0.023486
3,Elizabeth,1646385,0.009367
1412,Patricia,1572330,0.008945
6803,Jennifer,1468017,0.008352
304,Linda,1453065,0.008267


In [73]:
c2i = {}  # character => index
i2c = {}  # index => char

for index, char in enumerate(string.ascii_letters):
    c2i[char] = index
    i2c[index] = char

In [83]:
# upscale dataset to 250k names, weighted according to the normalized count
processed_dataset = []
for row in tqdm(df.itertuples()):
    name = row[1]
    pct = row[3]
    num_data_points = int(250000 * pct)
    index_series = np.array([c2i[char] / 52 for char in name]).astype('float32') # len(a-z + A-Z) = 52
    upscaled_data_for_name = [index_series for _ in range(num_data_points)]
    processed_dataset += upscaled_data_for_name

In [103]:
# shuffle the data
for _ in range(5):
    np.random.shuffle(processed_dataset)